# IBM Code Model Asset Exchange Human Pose Estimator

https://github.com/IBM/MAX-Human-Pose-Estimator

notes/links/etc

- `Netron`: https://github.com/lutzroeder/Netron
- `TF.js Converter`: https://github.com/tensorflow/tfjs-converter
- `TF.js API`: https://js.tensorflow.org/api/latest 


## Setup

1. In a terminal window, run the following commands to download and extract the model artifacts for the Human Pose Estimator:
    ```
    curl -O http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/human-pose-estimator/1.0/assets.tar.gz
    
    tar -zxvf assets.tar.gz
    ```    

2. Run the notebook `jupyter notebook .`

In [ ]:
# This notebook has been tested with Python version 3.6.6
!python --version

In [ ]:
# This notebook has been tested with tensorflow 1.12.0, tensorflowjs 0.8.0, numpy 1.16.1, opencv-python 3.4.4.19
!pip show tensorflow tensorflowjs numpy opencv-python

In [ ]:
# Uncomment to install the packages needed

# !pip install -Iv tensorflow
# !pip install -Iv tensorflowjs
# !pip install -Iv numpy
# !pip install -Iv opencv-python

# Restart the kernel after installation completes.

<br>
<strong>NOTE</strong>:

The post-processing for Part-Affinity Fields Map is implemented in C++ & Swig:

https://github.com/ildoonet/tf-pose-estimation/tree/master/tf_pose/pafprocess

To run locally on your machine:

- First install `swig`. For Mac OS X, you can install `swig` via `homebrew`

```
brew install swig
```

- Then build the `pafprocess` module:

```
swig -python -c++ pafprocess.i && python3 setup.py build_ext --inplace
```


<br>

Import libraries used in this notebook

In [ ]:
from PIL import Image
from matplotlib import pyplot as plt
import io
import pathlib
import os
import sys
import time
import numpy as np
import pprint
import cv2
import tensorflow as tf


<br>

Update the variable with the appropriate directory path

In [ ]:
# full path to cloned repo
human_pose_estimator = '/Users/va/machine-learning/MAX-Human-Pose-Estimator'

# full path to extracted frozen graph
frozen_graph_path = '/Users/va/machine-learning/human-pose-estimator-model/human-pose-estimator-tensorflow.pb'

# Add the repo to the Python path
sys.path.append(human_pose_estimator)


In [ ]:
print('TF versions:', tf.GIT_VERSION, tf.VERSION)

<br>

Set the helper functions

In [ ]:
import config

pp = pprint.PrettyPrinter(indent=2)

config.DEFAULT_MODEL_PATH = frozen_graph_path


def read_image(image_data):
    image = Image.open(io.BytesIO(image_data))
    image = np.array(image)[:,:,::-1] # Convert RGB to BGR for OpenCV
    return image


# Define the colors for different human body parts.
CocoColors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], 
              [170, 255, 0], [85, 255, 0], [0, 255, 0], [0, 255, 85], 
              [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], 
              [0, 0, 255], [85, 0, 255], [170, 0, 255], [255, 0, 255], 
              [255, 0, 170], [255, 0, 85]]


# Visualize the detected human poses on the image. The returned JSON result 
# contains the pose lines for each person in the input image. Each 
# person may have multiple pose lines. Each pose line contains four coordinates 
# for the start and end points as [x1, y1, x2, y2]. The `cv2.line(img, (x1, y1), 
# (x2, y2), color, thickness)` function is utilized to visualize the detected
# pose lines.
def draw_pose(humans, img):
    for human in humans:
        pose_lines = human['pose_lines']
        for i in range(len(pose_lines)):
            line = pose_lines[i]['line']
            cv2.line(img, (line[0], line[1]), (line[2], line[3]), CocoColors[i], 3)


<br>

Run prediction

In [ ]:
import core
from core.tf_pose.estimator import TfPoseEstimator

test_image_path = '/Users/va/machine-learning/images/img-03.jpg'

model = TfPoseEstimator(frozen_graph_path, target_size=config.DEFAULT_IMAGE_SIZE)

# run prediction on image
with open(test_image_path, 'rb') as image:
    img_array = read_image(image.read())
    humans = model.inference(img_array, resize_to_default=True, upsample_size=4.0)
    results = TfPoseEstimator.draw_human_pose_connection(img_array, humans)
    pp.pprint(results)


<br>

Visualize the detected poses

In [ ]:
# Visualize the detected poses overlaid on the original image.

org_img = cv2.imread(test_image_path)[:,:,::-1]

fig = plt.figure()
fig.set_size_inches(18.5, 10.5)

plt.subplot(1, 3, 1)
plt.imshow(org_img)
plt.title("The original image")

pose_img = np.zeros(org_img.shape, dtype=np.uint8)
draw_pose(results, pose_img)
plt.subplot(1, 3, 2)
plt.imshow(pose_img)
plt.title("The detected poses")

overlaid_img = org_img.copy()
draw_pose(results, overlaid_img)
plt.subplot(1, 3, 3)
plt.imshow(overlaid_img)
plt.title("Poses overlaid on original image")

plt.show()

<br>
<hr>

# Converting to a web-friendly format

[https://github.com/tensorflow/tfjs-converter](https://github.com/tensorflow/tfjs-converter)


```
tensorflowjs_converter \
    --input_format=tf_frozen_model \
    --output_node_names='Openpose/concat_stage7' \
    /path/to/frozen/model.pb \
    /path/to/web_asset_output_dir
```


Run converter

In [ ]:
# set appropriate desired output path for web format
web_asset_dir = '/Users/va/machine-learning/web-assets/human-pose-estimator'

# create directory if it does not exist
pathlib.Path(web_asset_dir).mkdir(parents=True, exist_ok=True)


In [ ]:

!tensorflowjs_converter \
    --input_format=tf_frozen_model \
    --output_node_names='Openpose/concat_stage7' \
    {frozen_graph_path} \
    {web_asset_dir}


In [ ]:
print("Web asset directory {}:".format(web_asset_dir))

web_assets = os.listdir(web_asset_dir)
web_assets.sort()

for file in web_assets:
    file_stat = os.stat("{}/{}".format(web_asset_dir,file))
    print(" {} {} {:>20}".format(file.ljust(30), time.ctime(file_stat.st_mtime), file_stat.st_size))
